In [2]:
#---Data---#
#Employees
employee = [:Manuel, :Luca, :Jule, :Michael, :Malte, :Chris, :Spyros,
            :Mirjam, :Matt, :Florian, :Josep, :Joel, :Tom, :Daniel, :Anne]

#Interview time slots
time = [1000, 1020, 1040, 1100, 1120, 1140, 1200, 1300, 1320, 1340, 1400, 1420, 1440]

#Schedule of availability(row: employees, column: time)
availability = 
           [0 0 1 1 0 0 0 1 1 0 0 0 0;
            0 1 1 0 0 0 0 0 1 1 0 0 0;
            0 0 0 1 1 0 1 1 0 1 1 1 1;
            0 0 0 1 1 1 1 1 1 1 1 1 0;
            0 0 0 0 0 0 1 1 1 0 0 0 0;
            0 1 1 0 0 0 0 0 1 1 0 0 0;
            0 0 0 1 1 1 1 0 0 0 0 0 0;
            1 1 0 0 0 0 0 0 0 0 1 1 1;
            1 1 1 0 0 0 0 0 0 1 1 0 0;
            0 0 0 0 0 0 0 1 1 0 0 0 0;
            0 0 0 0 0 0 1 1 1 0 0 0 0;
            1 1 0 0 0 1 1 1 1 0 0 1 1;
            1 1 1 0 1 1 0 0 0 0 0 1 1;
            0 1 1 1 0 0 0 0 0 0 0 0 0;
            1 1 0 0 1 1 0 0 0 0 0 0 0;]

#Number of employees that must meet for lunch
lunchEmployees = 3  

#Column index of lunch hour
lunchIndex = 7

#---Model---#
using JuMP
m = Model()

#The interview schedule
@defVar(m, schedule[1:length(employee), 1:length(time)] >= 0)

#There must be enough employees for lunch
@addConstraint(m, lunchNumber, sum{schedule[e, lunchIndex], e in 1:length(employee)} == lunchEmployees)

#There must be only one employee scheduled at each interview slot (before and after lunch hour)
@addConstraint(m, oneEmployee1[t in 1:(lunchIndex-1)], sum{schedule[e,t], e in 1:length(employee)} == 1)
@addConstraint(m, oneEmployee2[t in 8:length(time)], sum{schedule[e,t], e in 1:length(employee)} == 1)

#Only assign on available times
@addConstraint(m, onlyAvailable[e in 1:length(employee), t in 1:length(time)], 
availability[e, t] - schedule[e, t] >= 0)

#Assign each worker at least once
@addConstraint(m, onlyOnce[e in 1:length(employee)], sum{schedule[e, t], t in 1:length(time)} >= 1)

#Schedule by mimimizing assignments
@setObjective(m, Min, sum(schedule))

solve(m)
#Display schedule
println("Employees to meet: ")
for t in 1:length(time)
    print(time[t], ": ")
    
    for e in 1:length(employee)
        if(getValue(schedule[e,t]) == 1)
            print(employee[e], " ")
        end
    end
    println()
end

Employees to meet: 
1000: Anne 
1020: Chris 
1040: Manuel 
1100: Daniel 
1120: Spyros 
1140: Tom 
1200: Malte Josep Joel 
1300: Florian 
1320: Michael 
1340: Luca 
1400: Matt 
1420: Mirjam 
1440: Jule 
